In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

In [4]:
caffe.set_mode_cpu()
net = caffe.Net('../models/swimmers_fullconv/swimmers_fullconv.prototxt', 
                '../models/swimmers_fullconv/swimmers_fullconv.caffemodel', caffe.TEST)
net.reshape()

In [5]:
# find appearances weight blob
appearance_weights = net.params['appearances'][0].data
print appearance_weights.shape

(13, 392, 1, 1)


In [6]:
B = np.where(appearance_weights[2]==1)[0]
print len(B), B

0 []


In [7]:
appearance_path = '/data/share/idpr_code/appearanceclasses.txt'

def read_appearances(file): 
    with open(appearance_path) as f:
        content = f.readlines()

    joints = {}

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        joint = int(splitted[0])
        num = int(splitted[1])
        joints[joint] = map(int, splitted[2:2+num]) 

    return joints

In [8]:
joints = read_appearances(appearance_path)

for j in xrange(appearance_weights.shape[0]):
    for index in joints[j]:
        appearance_weights[j, index, 0, 0] = 1

In [9]:
B = np.where(appearance_weights[0]==1)[0]
print len(B), B

9 [  0  13  38  80  84 120 138 175 345]


In [29]:
idprclasses_path = '/data/share/idpr_code/idprclasses.txt'

def read_idprclasses(file): 
    with open(file) as f:
        content = f.readlines()

    classes = {}
    idx = 0
    #print content

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        from_ = int(splitted[0])
        to_ = int(splitted[1])
        cluster_ = int(splitted[2])
        n_ = int(splitted[3])
        classes[idx] = map(int, splitted[4:4+n_]) 
        idx+=1
        #print idx

    return classes

In [30]:
classes = read_idprclasses(idprclasses_path)

In [31]:
print len(classes)

152


In [32]:
# find idprclasses weight blob
idprclasses_weights = net.params['idprclasses'][0].data
print idprclasses_weights.shape

(152, 392, 1, 1)


In [35]:
classes = read_idprclasses(idprclasses_path)
#print classes

for j in xrange(idprclasses_weights.shape[0]):
    for index in classes[j]:
        idprclasses_weights[j, index, 0, 0] = 1


In [36]:
B = np.where(idprclasses_weights[7]==1)[0]
print len(B), B

90 [  7  14  19  28  34  43  49  67  71  76  81  89  98 103 110 113 115 118
 123 130 134 135 137 139 143 151 154 159 164 168 174 176 178 193 196 200
 207 208 209 212 213 218 221 222 228 239 240 241 244 246 250 256 269 272
 283 286 288 292 296 298 305 306 311 312 316 317 326 328 330 337 341 348
 349 351 352 353 354 356 360 361 364 365 368 369 371 372 373 375 378 380]


In [37]:
# save changes to new binaryproto
net.save('../models/swimmers_fullconv/out_new2.binaryproto')